In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
from functools import partial
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import callbacks
import os
import random
import pprint
import pandas as pd
import numpy as np

In [ ]:
df= pd.read_csv('/kaggle/input/nih-chest-xrays-tfrecords/preprocessed_data.csv')

In [ ]:
df.rename(columns = {'Unnamed: 0' : 'image'}, inplace = True)
df.head()

In [ ]:
columns = df.columns[1:]
plt.figure(figsize = (15,25))

for j,i in zip(range(len(columns)), range(1,16)):
            plt.subplot(4,4,i)
            valeur = df.value_counts(columns[j],sort = False)
            valeur.plot.bar(label =columns[j], color = {'green': 'False', 'red': 'True'})
            plt.legend();

In [ ]:
dossier = '/kaggle/input/nih-chest-xrays-tfrecords/data/'
image = os.listdir(dossier)
img = [dossier+x for x in image]
filenames = tf.io.gfile.glob(img)

In [ ]:
split_ind = int(0.8 * len(filenames))
split_ind2 = int(0.9 * len(filenames))

training= filenames[:split_ind]
valid = filenames[split_ind:split_ind2]
test = filenames[split_ind2:]

print("Train TFRecord Files:", len(training))
print("Validation TFRecord Files:", len(valid))
print("Test TFRecord Files:", len(test))
print(len(training)+len(valid)+len(test)==len(filenames))

In [ ]:
feature_map= {}

for elem in list(df.columns)[1:]:
    feature_map[elem] = tf.io.FixedLenFeature([], tf.int64)
    
feature_map['image'] = tf.io.FixedLenFeature([], tf.string)
feature_map

In [ ]:
def read_tfrecord(example):
    example = tf.io.parse_single_example(example, feature_map)
    image = tf.io.decode_jpeg(example["image"], channels=3)
    image = tf.image.resize(image, (150,150))
    image = tf.cast(image, tf.float32) / 255.0
    
    label = []
    
    for val in columns:
        label.append(example[val])
    
    return image, label

In [ ]:
def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_tfrecord)
    
    return dataset

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(64)
    
    return dataset

In [ ]:
train_dataset = get_dataset(training)
valid_dataset = get_dataset(valid)
test_dataset = get_dataset(test)

In [ ]:
eff = EfficientNetB1(include_top = False, input_shape=(150,150,3))
for layer in eff.layers:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(eff)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(15, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss ='binary_crossentropy',
              metrics = [tf.keras.metrics.AUC(name="auc")])

In [ ]:
plateau = callbacks.ReduceLROnPlateau(monitor='val_auc',
                                     patience = 1,
                                     factor = 0.01,
                                     mode = 'max')

early_stopping = callbacks.EarlyStopping(monitor = 'val_auc',
                                        patience = 3,
                                        restore_best_weights=True,
                                        mode = 'max')

In [ ]:
model.fit(train_dataset, epochs = 1,
          callbacks=[plateau, early_stopping], validation_data=valid_dataset)

In [ ]:
model.evaluate(test_dataset)